In [0]:
# Configuración de la clave de acceso a los contenedor de azure blob storage desde Databricks

spark.conf.set(
    "fs.azure.account.key.storagepredictsell.blob.core.windows.net", 
    "tR90WFDS7ts24Kr9Y1I9/Ptyy8xSLM6CgYoUTVmdMUZSDjG0Mn9o+/traZDmO1iguDysGtDH5YUg+AStGIr96A=="
)

# Define el nombre del contenedor y la cuenta de almacenamiento
container_name = "data"  # Nombre del contenedor en Azure
storage_account_name = "storagepredictsell"
storage_url = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"

# Listar los archivos en el contenedor de Azure Blob Storage
df = spark.read.format("binaryFile").load(storage_url)
display(df)


path,modificationTime,length,content
wasbs://data@storagepredictsell.blob.core.windows.net/test_blob.txt,2024-11-23T08:32:55Z,53,RXN0ZSBlcyB1biBhcmNoaXZvIGRlIHBydWViYSBwYXJhIEF6dXJlIEJsb2IgU3RvcmFnZS4=


In [0]:
dbutils.fs.ls(storage_url)


[FileInfo(path='wasbs://data@storagepredictsell.blob.core.windows.net/test_blob.txt', name='test_blob.txt', size=53, modificationTime=1732350775000)]

In [0]:
# Listar todos los puntos de montaje
dbutils.fs.ls('/mnt/storagepredictsell')



[FileInfo(path='dbfs:/mnt/storagepredictsell/data/', name='data/', size=0, modificationTime=1732451000000),
 FileInfo(path='dbfs:/mnt/storagepredictsell/kaggle/', name='kaggle/', size=0, modificationTime=1732451898000),
 FileInfo(path='dbfs:/mnt/storagepredictsell/test.txt', name='test.txt', size=29, modificationTime=1732452029000)]

In [0]:
%pip install kaggle

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
#IMPORTAR DATOS DE KAGGLE AL ENTORNO DE AZURE EN EL CONTENEDOR DEFINIDO 

# Importar las bibliotecas necesarias
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

# Configurar credenciales de Kaggle
os.environ['KAGGLE_USERNAME'] = 'juanp1018'  # Tu usuario de Kaggle
os.environ['KAGGLE_KEY'] = 'b9cb155d4b8231d90f0ae21780933769'  # Tu clave API de Kaggle

# Inicializar la API de Kaggle
api = KaggleApi()
api.authenticate()

print("Autenticación exitosa con la API de Kaggle.")

# Configurar las rutas
local_temp_path = "/tmp/kaggle"  # Ruta temporal para descargar el dataset
final_data_path = "/mnt/storagepredictsell/data"  # Ruta en DBFS para almacenar el dataset final

# Crear carpetas si no existen
os.makedirs(local_temp_path, exist_ok=True)
dbutils.fs.mkdirs(final_data_path)

# Descargar el dataset desde Kaggle
dataset_name = "mohammadtalib786/retail-sales-dataset"  # Nombre del dataset en Kaggle
api.dataset_download_files(dataset_name, path=local_temp_path, unzip=False)

print(f"Dataset descargado en: {local_temp_path}")

# Descomprimir el archivo ZIP en la carpeta temporal
zip_file_path = os.path.join(local_temp_path, "retail-sales-dataset.zip")
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(local_temp_path)

print(f"Archivo descomprimido en: {local_temp_path}")

# Listar los archivos descomprimidos para verificarlos
print("Archivos descomprimidos:")
extracted_files = os.listdir(local_temp_path)
for file_name in extracted_files:
    print(file_name)

# Mover archivos descomprimidos a la ruta de DBFS
for file_name in extracted_files:
    local_file_path = os.path.join(local_temp_path, file_name)
    dbfs_file_path = os.path.join(final_data_path, file_name)

    # Ignorar el archivo ZIP
    if not file_name.endswith(".zip"):
        with open(local_file_path, 'r') as file:
            file_content = file.read()
        dbutils.fs.put(dbfs_file_path, file_content, overwrite=True)

print(f"Archivos movidos a: {final_data_path}")

# Verificar los archivos en la carpeta final
print("Archivos disponibles en DBFS:")
files = dbutils.fs.ls(final_data_path)
for file in files:
    print(file.name)


Autenticación exitosa con la API de Kaggle.
Dataset URL: https://www.kaggle.com/datasets/mohammadtalib786/retail-sales-dataset
Dataset descargado en: /tmp/kaggle
Archivo descomprimido en: /tmp/kaggle
Archivos descomprimidos:
retail_sales_dataset.csv
retail-sales-dataset.zip
Wrote 50672 bytes.
Archivos movidos a: /mnt/storagepredictsell/data
Archivos disponibles en DBFS:
retail_sales_dataset.csv


In [0]:
# Verificar el contenido de la carpeta donde guardaste el dataset
files = dbutils.fs.ls("/mnt/storagepredictsell/data")
for file in files:
    print(file.name)


retail_sales_dataset.csv


In [0]:
# Cargar el dataset en un DataFrame de Spark
data_proyecto = spark.read.csv("/mnt/storagepredictsell/data/retail_sales_dataset.csv", header=True, inferSchema=True)

# Verificar las primeras filas del DataFrame
data_proyecto.show(5)


+--------------+----------+-----------+------+---+----------------+--------+--------------+------------+
|Transaction ID|      Date|Customer ID|Gender|Age|Product Category|Quantity|Price per Unit|Total Amount|
+--------------+----------+-----------+------+---+----------------+--------+--------------+------------+
|             1|2023-11-24|    CUST001|  Male| 34|          Beauty|       3|            50|         150|
|             2|2023-02-27|    CUST002|Female| 26|        Clothing|       2|           500|        1000|
|             3|2023-01-13|    CUST003|  Male| 50|     Electronics|       1|            30|          30|
|             4|2023-05-21|    CUST004|  Male| 37|        Clothing|       1|           500|         500|
|             5|2023-05-06|    CUST005|  Male| 30|          Beauty|       2|            50|         100|
+--------------+----------+-----------+------+---+----------------+--------+--------------+------------+
only showing top 5 rows



In [0]:
dbutils.fs.cp(
    "dbfs:/mnt/storagepredictsell/data/retail_sales_dataset.csv",
    "wasbs://data@storagepredictsell.blob.core.windows.net/retail_sales_dataset.csv"
)


True